In [1]:
import pandas as pd
from google.colab import files

uploaded =files.upload()

Saving entradas_breast.csv to entradas_breast.csv


In [2]:
import pandas as pd
import io
  
previsores = pd.read_csv(io.BytesIO(uploaded['entradas_breast.csv']))
previsores

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave_points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,1095.0000,0.9053,8589.0,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,0.5435,0.7339,3398.0,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.12380,0.18660,0.2416,186.0000,275.0000,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,0.7456,0.7869,4585.0,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.14440,0.42450,0.4504,243.0000,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,0.4956,1156.0000,3445.0,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,173.00000
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,198.00000,0.10430,0.1809,0.05883,0.7572,0.7813,5438.0,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.13740,205.00000,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,111.00000,0.11590,0.24390,0.13890,0.1726,0.05623,1176.0000,1256.0000,7673.0,158.70,0.010300,0.02891,0.05198,0.02454,0.01114,0.004239,25.45,26.40,166.10,2027.0,141.00000,0.21130,0.4107,0.2216,206.0000,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,144.00000,0.09791,0.1752,0.05533,0.7655,2463.0000,5203.0,99.04,0.005769,0.02423,0.03950,0.01678,0.01898,0.002498,23.69,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,159.0000,0.05648,0.4564,1075.0000,3425.0,48.55,0.005903,0.03731,0.04730,0.01557,0.01318,0.003892,18.98,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,277.00000,0.35140,152.00000,0.2397,0.07016,726.0000,1595.0000,5772.0,86.22,0.006522,0.06158,0.07117,0.01664,0.02324,0.006185,25.74,39.42,184.60,1821.0,165.00000,0.86810,0.9387,265.0000,0.4087,124.00000


In [3]:
uploaded =files.upload()

Saving saidas_breast.csv to saidas_breast.csv


In [4]:
classe = pd.read_csv(io.BytesIO(uploaded['saidas_breast.csv']))

In [25]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

def criarRede(optimizer, loss, kernel_initializer, activation, neurons):
  classificador=Sequential()
#Primeira camada oculta
#Definição da primeira camada de entrada (input_dim)
  classificador.add(Dense(units =neurons, activation= activation, kernel_initializer=kernel_initializer, input_dim=30)) 
#camada de dropout, passando o parametro da porcentagem que eu quero zerar
  classificador.add(Dropout(0.2))
#Segunda camada oculta  
  classificador.add(Dense(units =neurons, activation= activation, kernel_initializer=kernel_initializer)) 
  classificador.add(Dropout(0.2))
#camada de saida
  classificador.add(Dense(units =1, activation= 'sigmoid'))   
#classificador.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
  classificador.compile(optimizer=optimizer, loss=loss, metrics=['binary_accuracy'])
  return classificador

In [26]:
classificador = KerasClassifier(build_fn =criarRede)
parametros={'batch_size': [10,30],
            'epochs' : [50,100],
            'optimizer': ['adam','sgd'],
            'loss' : ['binary_crossentropy', 'hinge'],
            'kernel_initializer': ['random_uniform','normal'],
            'activation': ['relu', 'tanh'],
            'neurons': [16,8]}

In [27]:

grid_search=GridSearchCV(estimator=classificador,
                         param_grid=parametros, 
                         scoring='accuracy', 
                         cv=5)
grid_search=grid_search.fit(previsores,classe)
melhores_parametros=grid_search.best_params_
melhor_precisao=grid_search.best_score_

A saída de streaming foi truncada nas últimas 5000 linhas.
16/16 [==============================] - 1s 2ms/step - loss: 0.6806 - binary_accuracy: 0.5538
Epoch 2/100
16/16 [==============================] - 0s 4ms/step - loss: 0.6695 - binary_accuracy: 0.5978
Epoch 3/100
16/16 [==============================] - 0s 2ms/step - loss: 0.6735 - binary_accuracy: 0.6132
Epoch 4/100
16/16 [==============================] - 0s 2ms/step - loss: 0.6668 - binary_accuracy: 0.6242
Epoch 5/100
16/16 [==============================] - 0s 3ms/step - loss: 0.6653 - binary_accuracy: 0.6198
Epoch 6/100
16/16 [==============================] - 0s 2ms/step - loss: 0.6731 - binary_accuracy: 0.6396
Epoch 7/100
16/16 [==============================] - 0s 3ms/step - loss: 0.6663 - binary_accuracy: 0.6330
Epoch 8/100
16/16 [==============================] - 0s 3ms/step - loss: 0.6552 - binary_accuracy: 0.6418
Epoch 9/100
16/16 [==============================] - 0s 3ms/step - loss: 0.6542 - binary_accuracy: 0.6440

In [28]:
grid_search

GridSearchCV(cv=5, error_score=nan,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7fc0e8d79f10>,
             iid='deprecated', n_jobs=None,
             param_grid={'activation': ['relu', 'tanh'], 'batch_size': [10, 30],
                         'epochs': [50, 100],
                         'kernel_initializer': ['random_uniform', 'normal'],
                         'loss': ['binary_crossentropy', 'hinge'],
                         'neurons': [16, 8], 'optimizer': ['adam', 'sgd']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [29]:
melhores_parametros

{'activation': 'relu',
 'batch_size': 10,
 'epochs': 100,
 'kernel_initializer': 'random_uniform',
 'loss': 'binary_crossentropy',
 'neurons': 8,
 'optimizer': 'adam'}

In [31]:
melhor_precisao

0.9104486880919112